# Creating and adding a MetaCAT model to an existing MedCAT model

In order to get useful meta annotations, we need a separate model to make those predictions.
This tutorial will go through the steps to create and add a new `MetaCAT` model to an existing model pack.

First, we will need to install meta-cat specific parts to MedCAT.

In [ ]:
! pip install "git+https://github.com/CogStack/MedCAT2@v0.3.3#egg=medcat2[meta-cat]" # NOTE: VERSION-STRING

DEPRECATION: git+https://github.com/CogStack/MedCAT2@v0.1.5#egg=medcat2[meta-cat] contains an egg fragment with a non-PEP 508 name. pip 25.1 will enforce this behaviour change. A possible replacement is to use the req @ url syntax, and remove the egg fragment. Discussion can be found at https://github.com/pypa/pip/issues/13157
  Cloning https://github.com/CogStack/MedCAT2 (to revision v0.1.5) to /private/var/folders/h4/sklqg_zx1dbbbx76m2__zb8h0000gn/T/pip-install-3997cju_/medcat2_64f531cab4c3488a88bf357d0c17bab7
  Running command git clone --filter=blob:none --quiet https://github.com/CogStack/MedCAT2 /private/var/folders/h4/sklqg_zx1dbbbx76m2__zb8h0000gn/T/pip-install-3997cju_/medcat2_64f531cab4c3488a88bf357d0c17bab7
  Running command git checkout -q b1fe0ec5dc88ad01fb8dfeede0cd3c9144ff1443
  Resolved https://github.com/CogStack/MedCAT2 to commit b1fe0ec5dc88ad01fb8dfeede0cd3c9144ff1443
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing

We will assume that ou have ran through the creation of the model in the `basic` category.
And we will attempt to load the model you have saved there.
However, if you wish to do this with your own model, feel free to change the path as needed.

In [2]:
import os
from medcat2.cat import CAT
from medcat2.components.addons.meta_cat.meta_cat import MetaCATAddon
from medcat2.config.config_meta_cat import ConfigMetaCAT

BASIC_MODELS_FOLDER = os.path.join("..", "basic", "models")
MODEL_NAME = "sup_trained_model.zip"
MODEL_PATH = os.path.join(BASIC_MODELS_FOLDER, MODEL_NAME)

cat = CAT.load_model_pack(MODEL_PATH)

/Users/martratas/Documents/CogStack/.MedCAT.nosync/MedCATv2tutorials/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


We have a model, we need to create a new MetaCAT and add it

In [ ]:
# create config
config = ConfigMetaCAT()
# add categoryy name
config.general.category_name = 'Status'
config.model.nclasses = 3
config.general.category_value2id = {'False': 2, 'Hypothetical': 1, 'True': 0}

data_path = os.path.join("in_data", "tok_data.txt")

# create MetaCAT
# TODO: remove need for call to protected attribute
mc = MetaCATAddon.create_new(
    config, cat._pipeline.tokenizer,
    tknzer_preprocessor=lambda tknzer: tknzer.hf_tokenizers.train(data_path))

# add MetaCAT
if cat.config.components.addons:
    # avoid adding multiple on multiple runs
    print("Already had an addon, so not adding a 2nd (probably identical) one")
else:
    cat.add_addon(mc)
print("Addon configs:", cat.config.components.addons)
print("Addons", cat._pipeline._addons)




Addon configs: [ConfigMetaCAT(comp_name='default', init_args=[], init_kwargs={}, general=General(device='cpu', disable_component_lock=False, seed=13, description='No description', category_name='Status', category_value2id={'False': 2, 'Hypothetical': 1, 'True': 0}, vocab_size=15902, lowercase=True, cntx_left=15, cntx_right=10, replace_center=None, batch_size_eval=5000, tokenizer_name='bbpe', save_and_reuse_tokens=False, pipe_batch_size_in_chars=20000000, span_group=None, serialiser=<AvailableSerialisers.dill: 1>), model=Model(model_name='lstm', model_variant='bert-base-uncased', model_freeze_layers=True, num_layers=2, input_size=300, hidden_size=300, dropout=0.5, phase_number=0, category_undersample='', model_architecture_config={'fc2': True, 'fc3': False, 'lr_scheduler': True}, num_directions=2, nclasses=3, padding_idx=15901, emb_grad=True, ignore_cpos=False), train=Train(batch_size=100, nepochs=50, lr=0.001, test_size=0.1, shuffle_data=True, class_weights=None, compute_class_weigh

Now we have a MetaCAT added.
Let's see if we can see anything in the output.

In [6]:
from pprint import pprint
text_true = "The patient presents with a fluctuant abscess on the right thigh, consistent with a localized staphylococcal infection."
text_false = "On examination, there is no evidence of abscess or cellulitis around the wound site."
text_hypothetical = "If the swelling persists despite antibiotics, we should consider imaging to rule out a deep-seated abscess."
for text, expect in [(text_true, "True"), (text_false, "False"), (text_hypothetical, "Hypothetical")]:
    ents = cat.get_entities(text)["entities"]
    print("Expected", expect)
    pprint(ents)

Expected True
{0: {'acc': 0.99,
     'context_center': [],
     'context_left': [],
     'context_right': [],
     'context_similarity': 0.99,
     'cui': '44132006',
     'detected_name': 'abscess',
     'end': 45,
     'id': 0,
     'meta_anns': {'Status': {'confidence': 0.34161797165870667,
                              'name': 'Status',
                              'value': 'True'},
                   (5, 5): defaultdict(<function Entity.register_addon_path.<locals>.<lambda> at 0x138e1e4d0>,
                                       {'Status': {'confidence': 0.34161797165870667,
                                                   'name': 'Status',
                                                   'value': 'True'},
                                        (5, 5): <Recursion on defaultdict with id=5239348400>,
                                        (6, 6): <Recursion on defaultdict with id=5239348400>}),
                   (6, 6): defaultdict(<function Entity.register_addon_path.<local

As we can see, this doesn't yet work too well.
That's because our examples are extremely limited.